### Ньяти Каелиле БВТ2201
### Курсовая Работа - СИАОД

#### 1. Using LOB (Linear Optimization Based) Algorithm

In [ ]:
import pulp
from pulp import LpMinimize, LpProblem, LpVariable, lpSum, LpBinary, LpStatus
from pulp.apis import PULP_CBC_CMD
import pandas as pd

# Parameters
num_buses = 8
days_of_week = range(7)  # Days of the week (0: Monday, ..., 6: Sunday)
drivers_count = 10
num_8hr_drivers = drivers_count // 2  # Half of the drivers are 8-hour drivers
num_12hr_drivers = drivers_count - num_8hr_drivers
time_slots = range(24)  # Representing hours in a day

# Initialize the problem
model = LpProblem(name="bus-scheduling", sense=LpMinimize)

# Variables
x = LpVariable.dicts(
    "x", [(d, b, day, t) for d in range(drivers_count) 
          for b in range(num_buses) for day in days_of_week for t in time_slots],
    cat=LpBinary
)

# Constraints

# 1. Each bus must have one driver assigned to it during each hour of every day,
# ensuring that drivers are assigned to the same bus for their entire shift.
for b in range(num_buses):
    for day in days_of_week:
        for t in time_slots:
            # Check if a driver is assigned to this bus for this time slot
            model += lpSum(x[d, b, day, t] for d in range(drivers_count)) <= 1, f"Bus_{b}_Day_{day}_Hour_{t}_Coverage"

# 2. Ensure that if a driver is assigned to a bus at a given time, they stay on that bus
# for their entire shift duration. 
for d in range(drivers_count):
    for day in days_of_week:
        # Determine the shift duration for this driver
        shift_duration = 8 if d < num_8hr_drivers else 12
        
        for shift_start in range(0, 24 - shift_duration + 1):  # Loop through possible start times
            for b in range(num_buses):
                # If driver `d` is assigned to bus `b` at `shift_start`, they should stay on that bus for the full shift duration
                if any(x[d, b, day, t].varValue == 1 for t in range(shift_start, shift_start + shift_duration) if t < 24):
                    # Force the driver to stay on the same bus for the entire shift
                    for t in range(shift_start, shift_start + shift_duration):
                        if t < 24:
                            model += x[d, b, day, t] == 1, f"Driver_{d}_Day_{day}_Shift_{shift_start}_Bus_{b}_Hour_{t}"



# 3. 8-hour drivers work Monday to Friday and rest on weekends, sticking to one bus for the entire week
for d in range(num_8hr_drivers):
    for day in days_of_week:
        if day < 5:  # Monday to Friday
            for b in range(num_buses):
                for t in range(6, 14):  # Example: Shift from 06:00 to 14:00
                    model += x[d, b, day, t] == x[d, b, 0, 6], f"8hr_Driver_{d}_Bus_{b}_Day_{day}_Hour_{t}"
        else:  # Rest on weekends
            model += lpSum(x[d, b, day, t] for b in range(num_buses) for t in time_slots) == 0, f"8hr_Driver_{d}_Rest_Day_{day}"

# Ensure each 8-hour driver sticks to only one bus for the entire week
for d in range(num_8hr_drivers):
    model += lpSum(x[d, b, 0, 6] for b in range(num_buses)) == 1, print(f"8hr_Driver_{d}_Single_Bus_Assignment")

# 4. 12-hour drivers work 1 day and rest 2 consecutive days
for d in range(num_12hr_drivers, drivers_count):  # 12-hour drivers
    # Determine staggered start day based on driver index
    start_day = (d - num_12hr_drivers) % 3  # Stagger start days across 3 groups

    for day in days_of_week:
        if ((day - start_day) % 3 + 3) % 3 == 0:  # Work days depend on staggered start day
            for b in range(num_buses):
                # Afternoon Shift (14:00 - 24:00) - work on this later
                model += lpSum(x[d, b, day, t] for t in range(14, 24)) == 10, f"12hr_Driver_{d}_Day_{day}_Bus_{b}_Afternoon_Shift"

            
                # Night Shift: Continue working (00:00 to 02:00 on the next day)
                #model += lpSum(x[d, b, (day + 1) % 7, t] for t in range(0, 2)) == 2, f"12hr_Driver_{d}_Day_{(day + 1) % 7}_Bus_{b}_Night_Shift"

                # Night Shift (00:00 - 12:00) on the next day
                if day < 6:  # Ensure we don't exceed Sunday
                    model += lpSum(x[d, b, day + 1, t] for t in range(0, 12)) == 12, f"12hr_Driver_{d}_Day_{day+1}_Bus_{b}_Night_Shift"
        else:  # Rest days for drivers
            model += lpSum(x[d, b, day, t] for b in range(num_buses) for t in time_slots) == 0, f"12hr_Driver_{d}_Rest_Day_{day}"

# 5. Drivers cannot work on more than one bus at the same time
for d in range(drivers_count):
    for day in days_of_week:
        for t in time_slots:
            model += lpSum(x[d, b, day, t] for b in range(num_buses)) <= 1, f"Driver_{d}_Day_{day}_Hour_{t}_Single_Bus"

# 6. Ensure compact shifts for each driver (driver continuity on buses)
for d in range(drivers_count):
    for day in days_of_week:
        for b in range(num_buses):
            for t in time_slots[:-1]:  # Exclude the last time slot to prevent overflow
                # Ensure no gap between consecutive hour assignments for a driver
                model += x[d, b, day, t + 1] >= x[d, b, day, t], f"Driver_{d}_Bus_{b}_Day_{day}_Continuity_{t}"
                
# Objective: Minimize the number of active drivers (if needed)
model += lpSum(x[d, b, day, t] for d in range(drivers_count) 
               for b in range(num_buses) for day in days_of_week for t in time_slots), "MinimizeDrivers"


# Solve the model
status = model.solve(pulp.PULP_CBC_CMD(msg=1))

# Debugging: Output the solver status
print(f"Solver status: {LpStatus[status]}")

for v in model.variables():
    print(f"{v.name}: {v.varValue}")
# Define shift start times and durations
shift_start_times = {"8hr": "06:00", "12hr": "06:00"}
shift_durations = {"8hr": 8, "12hr": 12}

# Generate output schedule
# Define headers
days = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
driver_labels = [f"Driver {d + 1}" for d in range(drivers_count)]

# Initialize DataFrame
schedule_df = pd.DataFrame(index=driver_labels, columns=days)

# Populate DataFrame with Start/End Times and Bus Assignments
for d in range(drivers_count):
    shift_type = "8hr" if d < num_8hr_drivers else "12hr"
    start_time = shift_start_times[shift_type]
    duration = shift_durations[shift_type]
    end_time = f"{(int(start_time[:2]) + duration) % 24:02d}:00"  # Calculate end time

    for day in days_of_week:
        assigned_buses = [b for b in range(num_buses) if any(x[d, b, day, t].varValue == 1 for t in time_slots)]
        if assigned_buses:
            schedule_df.iloc[d, day] = f"Bus {', '.join(map(str, assigned_buses))} ({start_time}-{end_time})"
        else:
            schedule_df.iloc[d, day] = "Rest"

# Add grouping for 8-hour and 12-hour drivers
shift_types = ["8hr" if d < num_8hr_drivers else "12hr" for d in range(drivers_count)]
schedule_df.insert(0, "Shift Type", shift_types)

# Styling the DataFrame
styled_df = (
    schedule_df.style.set_properties(
        **{
            "text-align": "center",
            "font-weight": "bold",
            "border": "1px solid black",
            "padding": "5px",
            "font-size": "12px"
        }
    )
    .set_table_styles([ 
        {"selector": "th", "props": [("background-color", ""), ("font-size", "14px")]},
        {"selector": "td", "props": [("font-size", "12px"), ("padding", "5px")]},
    ])
    .set_caption("Bus Scheduling with Start and End Times")
)

# Display the styled DataFrame
styled_df


8hr_Driver_0_Single_Bus_Assignment
8hr_Driver_1_Single_Bus_Assignment
8hr_Driver_2_Single_Bus_Assignment
8hr_Driver_3_Single_Bus_Assignment
8hr_Driver_4_Single_Bus_Assignment
Solver status: Infeasible
x_(0,_0,_0,_0): 0.0
x_(0,_0,_0,_1): 0.0
x_(0,_0,_0,_10): 1.0
x_(0,_0,_0,_11): 1.0
x_(0,_0,_0,_12): 1.0
x_(0,_0,_0,_13): 1.0
x_(0,_0,_0,_14): 1.0
x_(0,_0,_0,_15): 1.0
x_(0,_0,_0,_16): 1.0
x_(0,_0,_0,_17): 1.0
x_(0,_0,_0,_18): 1.0
x_(0,_0,_0,_19): 1.0
x_(0,_0,_0,_2): 0.0
x_(0,_0,_0,_20): 1.0
x_(0,_0,_0,_21): 1.0
x_(0,_0,_0,_22): 1.0
x_(0,_0,_0,_23): 1.0
x_(0,_0,_0,_3): 0.0
x_(0,_0,_0,_4): 0.0
x_(0,_0,_0,_5): 0.0
x_(0,_0,_0,_6): 1.0
x_(0,_0,_0,_7): 1.0
x_(0,_0,_0,_8): 1.0
x_(0,_0,_0,_9): 1.0
x_(0,_0,_1,_0): 0.0
x_(0,_0,_1,_1): 0.0
x_(0,_0,_1,_10): 1.0
x_(0,_0,_1,_11): 1.0
x_(0,_0,_1,_12): 1.0
x_(0,_0,_1,_13): 1.0
x_(0,_0,_1,_14): 1.0
x_(0,_0,_1,_15): 1.0
x_(0,_0,_1,_16): 1.0
x_(0,_0,_1,_17): 1.0
x_(0,_0,_1,_18): 1.0
x_(0,_0,_1,_19): 1.0
x_(0,_0,_1,_2): 0.0
x_(0,_0,_1,_20): 1.0
x_(0,_0,_1,_21

,Shift Type,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday
Driver 1,8hr,Bus 0 (06:00-14:00),Bus 0 (06:00-14:00),Bus 0 (06:00-14:00),Bus 0 (06:00-14:00),Bus 0 (06:00-14:00),Rest,Rest
Driver 2,8hr,Bus 1 (06:00-14:00),Bus 1 (06:00-14:00),Bus 1 (06:00-14:00),Bus 1 (06:00-14:00),Bus 1 (06:00-14:00),Rest,Rest
Driver 3,8hr,Bus 5 (06:00-14:00),Bus 5 (06:00-14:00),Bus 5 (06:00-14:00),Bus 5 (06:00-14:00),Bus 5 (06:00-14:00),Rest,Rest
Driver 4,8hr,Rest,Rest,Rest,Rest,Rest,Rest,Rest
Driver 5,8hr,Rest,Rest,Rest,Rest,Rest,Rest,Rest
Driver 6,12hr,Bus 3 (06:00-18:00),Rest,Rest,Bus 4 (06:00-18:00),Rest,Rest,Bus 1 (06:00-18:00)
Driver 7,12hr,Rest,Bus 6 (06:00-18:00),Rest,Rest,Bus 2 (06:00-18:00),Rest,Rest
Driver 8,12hr,Rest,Rest,Bus 4 (06:00-18:00),Rest,Rest,Bus 0 (06:00-18:00),Rest
Driver 9,12hr,Bus 6 (06:00-18:00),Rest,Rest,Bus 2 (06:00-18:00),Rest,Rest,Bus 0 (06:00-18:00)
Driver 10,12hr,Rest,Bus 2 (06:00-18:00),Rest,Rest,Bus 7 (06:00-18:00),Rest,Rest
